<a href="https://colab.research.google.com/github/nagarajuerigi/Learning/blob/master/MNIST_Classf_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This code implements a machine learning workflow for classifying handwritten digits using the MNIST dataset and Streamlit for creating a web application.**

In [1]:
# Libraries
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [3]:
# Load MNIST data
mnist = fetch_openml('mnist_784', version=1, cache=True, as_frame=False)
print(mnist.DESCR)

X = mnist["data"]

y = mnist["target"].astype(np.uint8)

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


**Author**: Yann LeCun, Corinna Cortes, Christopher J.C. Burges  
**Source**: [MNIST Website](http://yann.lecun.com/exdb/mnist/) - Date unknown  
**Please cite**:  

The MNIST database of handwritten digits with 784 features, raw data available at: http://yann.lecun.com/exdb/mnist/. It can be split in a training set of the first 60,000 examples, and a test set of 10,000 examples  

It is a subset of a larger set available from NIST. The digits have been size-normalized and centered in a fixed-size image. It is a good database for people who want to try learning techniques and pattern recognition methods on real-world data while spending minimal efforts on preprocessing and formatting. The original black and white (bilevel) images from NIST were size normalized to fit in a 20x20 pixel box while preserving their aspect ratio. The resulting images contain grey levels as a result of the anti-aliasing technique used by the normalization algorithm. the images were centered in a 28x28 image b

In [4]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocess data (normalize)
X_train = X_train / 255.0
X_test = X_test / 255.0

In [5]:
# Define models
models = []
models.append(("Logistic Regression", LogisticRegression(multi_class="ovr", solver="lbfgs")))
models.append(("Random Forest", RandomForestClassifier(n_estimators=100)))

In [6]:
# Train and evaluate models
for name, model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{name} Accuracy: {accuracy:.2f}")


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression Accuracy: 0.92
Random Forest Accuracy: 0.97


In [7]:
# Choose the best model (based on accuracy in this example)
best_model_name, best_model = models[0]

# Evaluate best model on unseen data
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"\nBest Model ({best_model_name}) Accuracy on Unseen Data: {accuracy:.2f}")


Best Model (Logistic Regression) Accuracy on Unseen Data: 0.92


**Image Preprocessing Function**

In [8]:
import cv2

def preprocess_image(img):
  # Convert to grayscale
  img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

  # Resize to 28x28 pixels and normalize
  img_resized = cv2.resize(img_gray, (28, 28), interpolation=cv2.INTER_AREA)
  img_normalized = img_resized / 255.0

  # Reshape to match MNIST data format
  return img_normalized.reshape((1, -1))

**Streamlit Application**

In [10]:
pip install streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [11]:
import streamlit as st
# Load the best model (replace with your model loading logic)
model = best_model

# Title and header
st.title("MNIST Digit Classifier")
st.header("Predict handwritten digits using your webcam or uploaded image")

# Option 1: Webcam capture
if st.checkbox("Use Webcam"):
  # Get webcam frame
  frame = st.camera_input("Capture image")

  # Predict if a frame is available
  if frame is not None:
    # Convert frame to RGB and preprocess
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img_preprocessed = preprocess_image(img)

    # Make prediction
    prediction = model.predict(img_preprocessed)[0]

    # Display prediction with confidence score (optional)
    prediction_proba = model.predict_proba(img_preprocessed)[0]
    confidence = np.max(prediction_proba) * 100
    st.write(f"Predicted Digit: {prediction} (Confidence: {confidence:.2f}%)")
    st.image(img, channels="RGB", width=200)

# Option 2: Upload image
elif st.file_uploader("Upload Image", type=["jpg", "png"]):
  # Read uploaded image
  img = st.uploaded_file.read()
  img = cv2.imdecode(np.frombuffer(img, np.uint8), cv2.IMREAD_COLOR)

  # Predict
  img_preprocessed = preprocess_image(img)
  prediction = model.predict(img_preprocessed)[0]

  # Display prediction with confidence score (optional)
  prediction_proba = model.predict_proba(img_preprocessed)[0]
  confidence = np.max(prediction_proba) * 100
  st.write(f"Predicted Digit: {prediction} (Confidence: {confidence:.2f}%)")
  st.image(img, channels="RGB", width=200)

else:
  st.write("Select an option to use Webcam or upload an Image.")



2024-03-15 20:56:30.015 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
